In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import plotly.express as px

import yfinance as yf

from portfolio_dash.pflib import *

In [2]:
pd.set_option('display.precision',4)

In [3]:
def generate_portfolios(returns, numPortfolios, riskFreeRate=0, riskFreeAsset=False, shortSelling=False):
    tickers = returns.columns.tolist()
    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252

    # If the risk-free asset is included in the portfolio, add it to mean_returns and cov_matrix
    if riskFreeAsset:
        tickers = tickers + ['RF']
        # Concatenate the risk-free rate to the mean_returns
        mean_returns = np.append(mean_returns, riskFreeRate)
        # Expand the covariance matrix to include the risk-free rate
        cov_matrix = np.vstack([cov_matrix, np.zeros(nAssets-1)])
        cov_matrix = np.hstack([cov_matrix, np.zeros(nAssets).reshape(-1,1)])

    nAssets = len(tickers)
    
    # Create an empty DataFrame to store the results
    portfolios = pd.DataFrame(columns=[ticker+' weight' for ticker in tickers] + ['Return', 'Risk', 'Sharpe Ratio'], index=range(numPortfolios), dtype=float)

    # Generate random weights and calculate the expected return, volatility and Sharpe ratio
    for i in range(numPortfolios):
        weights = np.random.random(nAssets)
        weights /= np.sum(weights)
        portfolios.loc[i, [ticker+' weight' for ticker in tickers]] = weights

        # Calculate the expected return
        portfolios.loc[i, 'Return'] = np.dot(weights, mean_returns)

        # Calculate the expected volatility
        portfolios.loc[i, 'Risk'] = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Calculate the Sharpe ratio
    portfolios['Sharpe Ratio'] = (portfolios['Return'] - riskFreeRate) / portfolios['Risk']

    return portfolios
    
def evaluate_portfolio(mc_portfolios, index, data, initialValue):
    portfolio = mc_portfolios.loc[index]
    tickers = data.columns
    nShares = portfolio[[ticker+' weight' for ticker in tickers]].rename({ticker+' weight' : ticker for ticker in tickers})*initialValue/data.iloc[0]
    portfolio_value = nShares.dot(data.T)
    return portfolio_value

def evaluate_asset(tickers, index, data, initialValue):
    asset = data.iloc[:, index] if len(tickers) > 1 else data
    nShares = initialValue/asset.iloc[0]
    asset_value = nShares*asset
    return asset_value

def minimum_variance_portfolio(mean_returns, cov_matrix):
    nAssets = len(mean_returns)
    m = mean_returns
    C = cov_matrix
    u = np.ones(nAssets)

    # Intermediate calculations
    C_inv = np.linalg.inv(C)
    w = C_inv.dot(u) / u.T.dot(C_inv).dot(u)
    mu = m.T.dot(w)
    sigma = np.sqrt(w.T.dot(C).dot(w))

    return mu, sigma

def minimum_variance_line(mean_returns, cov_matrix):
    nAssets = len(mean_returns)
    m = mean_returns
    C = cov_matrix
    u = np.ones(nAssets)

    # Intermediate calculations
    C_inv = np.linalg.inv(C)
    D_mat = np.array([[u.T.dot(C_inv).dot(u), u.T.dot(C_inv).dot(m)],
                      [m.T.dot(C_inv).dot(u), m.T.dot(C_inv).dot(m)]])
    D = np.linalg.det(D_mat)
    a = (u.T.dot(C_inv).dot(u)*C_inv.dot(m) - u.T.dot(C_inv).dot(m)*C_inv.dot(u)) / D
    b = (m.T.dot(C_inv).dot(m)*C_inv.dot(u) - m.T.dot(C_inv).dot(u)*C_inv.dot(m)) / D

    # Calculate the minimum variance line
    mu = np.linspace(0, 0.4, 100)
    w_mu = np.zeros((nAssets, len(mu)))
    sigma_mu = np.zeros(len(mu))
    for i, param in enumerate(mu):
        w_mu[:,i] = a*param + b
        sigma_mu[i] = np.sqrt(w_mu[:,i].T.dot(C).dot(w_mu[:,i]))

    return mu, sigma_mu

def market_portfolio(mean_returns, cov_matrix, riskFreeRate=0):
    nAssets = len(mean_returns)
    m = mean_returns
    C = cov_matrix
    u = np.ones(nAssets)

    # Intermediate calculations
    C_inv = np.linalg.inv(C)
    w = C_inv.dot(m - riskFreeRate*u) / (m - riskFreeRate*u).T.dot(C_inv).dot(u)
    mu = m.T.dot(w)
    sigma = np.sqrt(w.T.dot(C).dot(w))

    return mu, sigma

    

In [4]:
# Get a list of symbols from FTSEMIB index
ftsemib = pd.read_html('https://en.wikipedia.org/wiki/FTSE_MIB')[1]
ftsemib['ICB Sector'] = ftsemib['ICB Sector'].str.extract(r'\((.*?)\)', expand=False).fillna(ftsemib['ICB Sector'])

## Get list of assets

In [5]:
tickers = ['ENI.MI', 'ENEL.MI', 'STMMI.MI', 'LDO.MI', 'A2A.MI',]
tickers = sorted(tickers)
riskFreeRate = 0.04


In [6]:
start = dt.date.today() - dt.timedelta(days=2*365)
end = dt.date.today()

myBasket = Basket(tickers, riskFreeRate)
myBasket.get_data(start=start, end=end)


data = yf.download(tickers, start=start, end=end, progress=False )['Adj Close']

In [7]:
# Split the data into in-sample and out-of-sample
sample_end = dt.date.today() - dt.timedelta(days=365)
inSample_data = data[:sample_end]
outOfSample_data = data[sample_end:]

In [8]:
# Get the daily returns
returns = inSample_data.pct_change().dropna()

# Collect drifts and standard deviations in the columns of a single DataFrame
tickers_df = pd.DataFrame({'Return': returns.mean() * 252, 'Risk': np.sqrt(np.diag(returns.cov() * 252))}, index=tickers).rename_axis('Ticker')
tickers_df.loc['Risk Free Rate'] = [riskFreeRate, 0]


In [9]:
tickers_df

,Return,Risk
Ticker,,
A2A.MI,-0.0248,0.2986
ENEL.MI,-0.0585,0.2703
ENI.MI,0.0341,0.3038
LDO.MI,0.3383,0.3474
STMMI.MI,0.2919,0.3894
Risk Free Rate,0.0400,0.0000


In [10]:
myBasket.mv_analysis(sample_end)[0]

,Return,Risk
A2A.MI,-0.0248,0.2986
ENEL.MI,-0.0585,0.2703
ENI.MI,0.0341,0.3038
LDO.MI,0.3383,0.3474
STMMI.MI,0.2919,0.3894


In [11]:
np.sqrt(np.diag(myBasket.data[:sample_end].pct_change().dropna().cov() * 252))

array([0.2986347 , 0.27031393, 0.30379976, 0.3474027 , 0.3894404 ])

In [12]:
inSample_data.pct_change().dropna().var().apply(np.sqrt) * np.sqrt(252)

Ticker
A2A.MI      0.2986
ENEL.MI     0.2703
ENI.MI      0.3038
LDO.MI      0.3474
STMMI.MI    0.3894
dtype: float64

In [13]:
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

# # Concatenate the risk-free rate to the mean_returns
# mean_returns = np.append(mean_returns, riskFreeRate)
# cov_matrix = np.append(cov_matrix, [np.zeros(nAssets-1)], axis=0)
# cov_matrix = np.hstack([cov_matrix, np.zeros(nAssets).reshape(-1,1)])

# Minimum variance line
mu, sigma_mu = minimum_variance_line(mean_returns, cov_matrix)
# Minimum variance portfolio
min_var_mu, min_var_sigma = minimum_variance_portfolio(mean_returns, cov_matrix)
# Market portfolio
market_mu, market_sigma = market_portfolio(mean_returns, cov_matrix, riskFreeRate)

Add minimum variance line too

In [14]:
# Plot the mean return and standard deviation with plotly
fig = px.scatter(tickers_df, x='Risk', y='Return', text=tickers_df.index)
fig.update_traces(textposition='top center')
fig.update_layout(title='Mean return vs Standard deviation', xaxis_title='Standard deviation', yaxis_title='Mean return')
fig.add_scatter(x=sigma_mu, y=mu, mode='lines', name='Minimum variance line')
fig.add_scatter(x=[min_var_sigma], y=[min_var_mu], mode='markers', name='Minimum variance portfolio', marker=dict(size=10, color='red'))
fig.add_scatter(x=[market_sigma], y=[market_mu], mode='markers', name='Market portfolio', marker=dict(size=10, color='green'))
fig.add_scatter(x=[0,market_sigma], y=[riskFreeRate,market_mu], mode='lines', name='Capital market line', line=dict(color='green',))
fig.show()


In [15]:
numPortfolios = 10000

mc_portfolios = generate_portfolios(returns, numPortfolios, riskFreeRate=riskFreeRate)

# Plot the portfolios
fig = px.scatter(mc_portfolios, x='Risk', y='Return', color='Sharpe Ratio',  hover_data={**{ticker +' weight': ':.2f' for ticker in tickers}, **{'Return': ':.2f', 'Risk': ':.2f', 'Sharpe Ratio': ':.2f'}}, opacity=0.5,)

fig.update_layout(title='Portfolios', xaxis_title='Standard deviation', yaxis_title='Mean return')
fig.add_scatter(x=tickers_df['Risk'], y=tickers_df['Return'], mode='markers', marker=dict(size=7.5, color='black',),showlegend=False, name='Tickers', text = [f'<b>{index}</b> <br>Standard deviation: {vol:.2f}<br>Expected return: {ret:.2f}' for index, vol, ret in zip(tickers_df.index, tickers_df['Risk'], tickers_df['Return'])],hoverinfo='text')
fig.add_scatter(x=sigma_mu, y=mu, mode='lines', name='Minimum variance line')
fig.show()


In [16]:
# Select sample portfolio
index = 256
sample_portfolio = mc_portfolios.loc[index]
# Determine the performance of the portfolio with out-of-sample data
initialValue = 100
samplePortfolio_value = evaluate_portfolio(mc_portfolios, index, outOfSample_data, initialValue)

fig = px.line(samplePortfolio_value, title='Portfolio value')
fig.update_layout(yaxis_title='Portfolio value')
fig.show()


Compare in-sample and out-of-sample returns

In [17]:
rolling_rets = data.pct_change().dropna().rolling(252).mean().dropna()*252
rolling_rets

Ticker,A2A.MI,ENEL.MI,ENI.MI,LDO.MI,STMMI.MI
Date,,,,,
2023-02-22,-0.0169,-0.0525,0.1022,0.3186,0.2730
2023-02-23,0.0093,-0.0479,0.0291,0.3433,0.2895
2023-02-24,0.0364,-0.0180,0.0624,0.3469,0.2831
2023-02-27,0.0608,0.0437,0.1512,0.4151,0.3738
2023-02-28,0.0601,0.0649,0.1057,0.3512,0.3833
...,...,...,...,...,...
2024-02-22,0.2812,0.2050,0.1632,0.6508,0.0060
2024-02-23,0.2692,0.1954,0.1492,0.6318,-0.0320
2024-02-26,0.2563,0.1796,0.1515,0.6844,-0.0330


In [18]:
# Create a new column in data and assign quarter and year to each row
rolling_rets['Quarter'] = rolling_rets.index.to_period('Q')
rolling_rets

Ticker,A2A.MI,ENEL.MI,ENI.MI,LDO.MI,STMMI.MI,Quarter
Date,,,,,,
2023-02-22,-0.0169,-0.0525,0.1022,0.3186,0.2730,2023Q1
2023-02-23,0.0093,-0.0479,0.0291,0.3433,0.2895,2023Q1
2023-02-24,0.0364,-0.0180,0.0624,0.3469,0.2831,2023Q1
2023-02-27,0.0608,0.0437,0.1512,0.4151,0.3738,2023Q1
2023-02-28,0.0601,0.0649,0.1057,0.3512,0.3833,2023Q1
...,...,...,...,...,...,...
2024-02-22,0.2812,0.2050,0.1632,0.6508,0.0060,2024Q1
2024-02-23,0.2692,0.1954,0.1492,0.6318,-0.0320,2024Q1
2024-02-26,0.2563,0.1796,0.1515,0.6844,-0.0330,2024Q1


In [19]:
# Calculate mean returns and standard deviations for each quarter
quarterly_returns = rolling_rets.groupby('Quarter').mean()
quarterly_returns


Ticker,A2A.MI,ENEL.MI,ENI.MI,LDO.MI,STMMI.MI
Quarter,,,,,
2023Q1,0.0125,0.0097,0.0859,0.2748,0.2831
2023Q2,0.1575,0.1318,0.1242,0.1560,0.2879
2023Q3,0.4842,0.3263,0.3115,0.4395,0.3006
2023Q4,0.5083,0.3357,0.2221,0.6635,0.2471
2024Q1,0.3293,0.2196,0.1285,0.6488,0.0448


In [20]:
quarterly_vol = rolling_rets.groupby('Quarter').std()

In [21]:
df_dict = {}
for stock in quarterly_returns.columns:
    df = pd.DataFrame({'Return': quarterly_returns[stock], 'Risk': quarterly_vol[stock]}, index=quarterly_returns.index)
    df_dict[stock] = df

df_dict['ENI.MI']


,Return,Risk
Quarter,,
2023Q1,0.0859,0.0430
2023Q2,0.1242,0.0722
2023Q3,0.3115,0.0589
2023Q4,0.2221,0.0707
2024Q1,0.1285,0.0418


In [22]:
px.line(df_dict['LDO.MI'], x='Risk', y='Return', hover_name=df_dict['LDO.MI'].index.strftime('%Y-Q%q'))

In [23]:
df_dict['LDO.MI'].index[1]

Period('2023Q2', 'Q-DEC')

In [24]:
# Convert period object to string showing only the year and quarter
df_dict['LDO.MI'].index = df_dict['LDO.MI'].index.strftime('%Y-Q%q')

In [25]:
df_dict['LDO.MI']

,Return,Risk
Quarter,,
2023-Q1,0.2748,0.0598
2023-Q2,0.1560,0.0453
2023-Q3,0.4395,0.1870
2023-Q4,0.6635,0.0407
2024-Q1,0.6488,0.0689


In [27]:
import json

portfolio = Portfolio(myBasket, 0.04, includeRiskFree=False)
portfolio

Portfolio(Basket containing tickers: [Stock(A2A.MI), Stock(ENEL.MI), Stock(ENI.MI), Stock(LDO.MI), Stock(STMMI.MI)], r = 0.04, )

In [29]:
portfolio.__dict__

{'basket': Basket([Stock(A2A.MI), Stock(ENEL.MI), Stock(ENI.MI), Stock(LDO.MI), Stock(STMMI.MI)]),
 'riskFreeRate': 0.04,
 'includeRiskFree': False}

In [28]:
json.dumps(portfolio, )

TypeError: Object of type Portfolio is not JSON serializable

In [34]:
mc_portfolios.iloc[2304][['ENI.MI weight', 'ENEL.MI weight', 'STMMI.MI weight', 'LDO.MI weight', 'A2A.MI weight']].rename()

ENI.MI weight      0.4064
ENEL.MI weight     0.1813
STMMI.MI weight    0.0306
LDO.MI weight      0.1750
A2A.MI weight      0.2067
Name: 2304, dtype: float64

In [42]:
r = 0.01
pd.Series((1+r)**(np.arange(len(data))/252),index = data.index)

Date
2022-03-01    1.0000
2022-03-02    1.0000
2022-03-03    1.0001
2022-03-04    1.0001
2022-03-07    1.0002
               ...  
2024-02-22    1.0202
2024-02-23    1.0202
2024-02-26    1.0203
2024-02-27    1.0203
2024-02-28    1.0203
Length: 511, dtype: float64